In [24]:
import sys
import pandas as pd
import backtrader as bt
from loguru import logger
%load_ext autoreload
%autoreload 2
%matplotlib inline

logger.remove()
logger.add(sys.stderr, level="INFO")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


6

In [25]:
import datetime

dateparse = lambda x: datetime.datetime.strptime(x, '%Y-%m-%d %H.%M.%S')
data = pd.read_csv("data.csv", parse_dates=['timestamp'], index_col='idx', date_parser=dateparse)
strategy_data = data.resample(rule='15T', on='timestamp').agg(
    {'open': 'first',
     'high': 'max',
     'low': 'min',
     'close': 'last',
     'volume': 'sum'
     })
strategy_data = strategy_data.reset_index()
strategy_data.head(5)

,timestamp,open,high,low,close,volume
0,2022-12-01 00:00:00,17165.44,17194.00,17133.35,17188.29,3008
1,2022-12-01 00:15:00,17188.85,17236.29,17159.61,17202.17,5286
2,2022-12-01 00:30:00,17202.17,17210.53,17151.02,17155.80,2505
3,2022-12-01 00:45:00,17155.80,17179.38,17122.65,17161.55,2565
4,2022-12-01 01:00:00,17161.55,17170.32,17134.07,17145.88,2172


In [27]:
from Bbands import StFetcher, BbandsStrategy
from backtrader.feeds import PandasData

bt_data = PandasData(dataname=strategy_data,
                     datetime='timestamp',
                     high='high',
                     low='low',
                     open='open',
                     close='close',
                     volume='volume')
cerebro = bt.Cerebro()
broker = cerebro.broker
broker.setcash(100000.0)
cerebro.adddata(bt_data)
cerebro.optstrategy(StFetcher,period=[20,50], bias=[1.1,1.2])
%timeit cerebro.run()


2023-01-18 21:37:51.477 | INFO     | Bbands:__init__:13 - parameter is 20-1.1
2023-01-18 21:37:51.477 | INFO     | Bbands:__init__:13 - parameter is 20-1.2
2023-01-18 21:37:51.477 | INFO     | Bbands:__init__:13 - parameter is 50-1.2
2023-01-18 21:37:51.477 | INFO     | Bbands:__init__:13 - parameter is 50-1.1


945 ms ± 35.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [28]:
cerebro2 = bt.Cerebro()
cerebro2.adddata(bt_data)
cerebro2.optstrategy(BbandsStrategy,period=[20,50], bias=[1.1,1.2])
%timeit cerebro2.run()


1.82 s ± 61.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [29]:
cerebro3 = bt.Cerebro()
cerebro3.adddata(bt_data)
cerebro3.addstrategy(BbandsStrategy,period=20, bias=1.1)
cerebro3.addstrategy(BbandsStrategy,period=20, bias=1.2)
cerebro3.addstrategy(BbandsStrategy,period=50, bias=1.1)
cerebro3.addstrategy(BbandsStrategy,period=50, bias=1.2)

%timeit cerebro3.run()


2023-01-18 21:40:38.876 | INFO     | Bbands:__init__:13 - parameter is 20-1.1
2023-01-18 21:40:38.879 | INFO     | Bbands:__init__:13 - parameter is 20-1.2
2023-01-18 21:40:38.882 | INFO     | Bbands:__init__:13 - parameter is 50-1.1
2023-01-18 21:40:38.884 | INFO     | Bbands:__init__:13 - parameter is 50-1.2
2023-01-18 21:40:40.505 | INFO     | Bbands:__init__:13 - parameter is 20-1.1
2023-01-18 21:40:40.508 | INFO     | Bbands:__init__:13 - parameter is 20-1.2
2023-01-18 21:40:40.511 | INFO     | Bbands:__init__:13 - parameter is 50-1.1
2023-01-18 21:40:40.514 | INFO     | Bbands:__init__:13 - parameter is 50-1.2
2023-01-18 21:40:42.190 | INFO     | Bbands:__init__:13 - parameter is 20-1.1
2023-01-18 21:40:42.193 | INFO     | Bbands:__init__:13 - parameter is 20-1.2
2023-01-18 21:40:42.196 | INFO     | Bbands:__init__:13 - parameter is 50-1.1
2023-01-18 21:40:42.199 | INFO     | Bbands:__init__:13 - parameter is 50-1.2
2023-01-18 21:40:43.958 | INFO     | Bbands:__init__:13 - parame

1.73 s ± 62 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
